In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

citydata = pd.read_csv("output_data/cities.csv")
citydata.head()

,Unnamed: 0,Cityname,Cloudiness,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Country,Date
0,0,East London,0,-33.02,27.91,48.20,70,8.05,ZA,1597374995
1,1,Kiryat Gat,0,31.61,34.76,75.99,42,3.44,IL,1597374996
2,2,Cape Town,20,-33.93,18.42,52.00,93,4.70,ZA,1597374901
3,3,Arraial do Cabo,0,-22.97,-42.03,71.96,84,26.69,BR,1597374702
4,4,Bethel,40,41.37,-73.41,72.00,94,1.81,US,1597374999


In [2]:
#clean up data, remove unamed column and rest index
citydata = citydata.reset_index().drop(columns = ["Unnamed: 0", "index"])
citydata

,Cityname,Cloudiness,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Country,Date
0,East London,0,-33.02,27.91,48.20,70,8.05,ZA,1597374995
1,Kiryat Gat,0,31.61,34.76,75.99,42,3.44,IL,1597374996
2,Cape Town,20,-33.93,18.42,52.00,93,4.70,ZA,1597374901
3,Arraial do Cabo,0,-22.97,-42.03,71.96,84,26.69,BR,1597374702
4,Bethel,40,41.37,-73.41,72.00,94,1.81,US,1597374999
...,...,...,...,...,...,...,...,...,...
559,Gorontalo,26,0.54,123.06,88.90,47,7.18,ID,1597375820
560,Vaitape,9,-16.52,-151.75,79.27,81,17.16,PF,1597375821
561,Olinda,40,-8.01,-34.86,75.20,78,8.05,BR,1597375823
562,Assiut,0,27.18,31.18,78.80,50,9.17,EG,1597375824


In [3]:
# filtering citydata for vacation criteria: lower than 80 but higher tha 70, wind speed 10 and no cloudiess
#use loc to filter and find exact locations that meet my criteria
#drop = True to remove the old index

vacationdata = citydata.loc[(citydata["Max Temp"] >70) & (citydata["Max Temp"] <80) & (citydata["Wind Speed"]<10) & (citydata["Cloudiness"] == 0)]
vacationdata= vacationdata.reset_index(drop = True)
vacationdata

,Cityname,Cloudiness,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Country,Date
0,Kiryat Gat,0,31.61,34.76,75.99,42,3.44,IL,1597374996
1,Albany,0,42.60,-73.97,73.00,68,3.87,US,1597374776
2,Erzin,0,36.96,36.20,77.00,90,1.72,TR,1597375068
3,Jaciara,0,-15.97,-54.97,74.34,41,2.28,BR,1597375106
4,Vila Velha,0,-20.33,-40.29,71.60,78,6.93,BR,1597375327
5,São Miguel do Araguaia,0,-13.28,-50.16,74.57,33,4.18,BR,1597375344
6,Mandalgovi,0,45.76,106.27,70.36,43,2.17,MN,1597375484
7,Erenhot,0,43.65,111.98,76.24,39,3.36,CN,1597375517
8,Jalu,0,29.03,21.55,75.38,51,7.29,LY,1597375531
9,Awjilah,0,29.11,21.29,75.90,45,7.07,LY,1597375639


In [4]:
#creating a heat map for the vacation citie, using the google maps API
# Configure gmaps with API key
# tells gmaps to use our API key
gmaps.configure(api_key=g_key)

# creating a list to feed the application API
citycoords = vacationdata[["Latitude", "Longitude"]].astype(float)
humidity = vacationdata["Humidity"].astype(float)

In [26]:
heatmap = gmaps.figure()
heatlayer = gmaps.heatmap_layer(citycoords, weights= humidity,
                                dissipating=False, max_intensity=90,
                                point_radius = 1)

heatmap.add_layer(heatlayer)
heatmap

Figure(layout=FigureLayout(height='420px'))

In [11]:
#creating a list called hotelnames to put a list of hotels found by the google places API searching for hotels within 5000 meetings
#of the longititude and latitude of each vacation location.

#These are the options we are presenting to the google place api for each call
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}


hotelnames = []


#For each row in our vacationdata dataframe get the the lat and lng for each location and add it to our params dictionary
#and make unique query based on the options defined

for index, row in vacationdata.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"


    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and convert response to json
    hotelinfo = requests.get(base_url, params=params).json()

 #if there is data in the name column of the hotelinfo json add it to the hotelnames list. 
    try:
        hotelnames.append(hotelinfo["results"][0]["name"])
 #if there is no data add no hotel found   
    except:
        hotelnames.append("No hotel found")

    

In [16]:
vacationdata["Hotel Name"] = hotelnames
vacationdata_whotel = vacationdata.loc[vacationdata["Hotel Name"] != "No hotel found"]


In [17]:
#to get the map to work we need a data fram with hotel name, that is the closest hotel to the cities
vacationdata_whotel

,Cityname,Cloudiness,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Country,Date,Hotel Name
0,Kiryat Gat,0,31.61,34.76,75.99,42,3.44,IL,1597374996,Desert Gat
2,Erzin,0,36.96,36.20,77.00,90,1.72,TR,1597375068,Hattusa Vacation Thermal Club Erzin
3,Jaciara,0,-15.97,-54.97,74.34,41,2.28,BR,1597375106,Hotel Toquinho
4,Vila Velha,0,-20.33,-40.29,71.60,78,6.93,BR,1597375327,Hotel Senac Ilha do Boi
5,São Miguel do Araguaia,0,-13.28,-50.16,74.57,33,4.18,BR,1597375344,Hotel Veredas do Araguaia
6,Mandalgovi,0,45.76,106.27,70.36,43,2.17,MN,1597375484,Mandal Hotel
7,Erenhot,0,43.65,111.98,76.24,39,3.36,CN,1597375517,Moli International Hotel
8,Jalu,0,29.03,21.55,75.38,51,7.29,LY,1597375531,Jalu Hotel
11,Damietta,0,31.42,31.81,78.80,83,5.82,EG,1597375792,SULIMAN INN
12,Nador,0,35.17,-2.93,73.40,40,1.12,MA,1597375819,"Hotel Marchica Lagoon Resort, Nador Morocco"


In [23]:

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cityname}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotelinfo = [info_box_template.format(**row) for index, row in vacationdata_whotel.iterrows()]
locations = vacationdata_whotel[["Latitude", "Longitude"]]

In [25]:
marker_layer = gmaps.marker_layer(locations, info_box_content = hotelinfo)
heatmap.add_layer(marker_layer)
heatmap

Figure(layout=FigureLayout(height='420px'))